<a href="https://colab.research.google.com/github/KhangTheKangaroo/Kanga-Bot/blob/main/Kanga's_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!gdown --id 1wBgErK1m7SzdxoZmjKWNPn1Rgdcg8VWS
!unzip -q Data.zip
!pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1wBgErK1m7SzdxoZmjKWNPn1Rgdcg8VWS
To: /content/Data.zip
100% 2.94k/2.94k [00:00<00:00, 8.66MB/s]
replace shop.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Database.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace requirements.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Help.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
  Using cached asyncio-3.4.3-py3-none-any.whl.metadata (1.7 kB)
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
from discord.ext import commands, tasks
import yt_dlp as ytdlp
import discord
import asyncio
import json
import os
import io
import urllib.parse
import urllib.request
import re
from discord.ui import Button, View
import datetime
from datetime import timedelta, datetime, timezone
import random as rand
import matplotlib.pyplot as plt
import time as t
import nest_asyncio

# Retrieve the Discord bot token and OpenAI API key from environment variables.
discord_bot_token = "[YOUR_DISCORD_BOT_TOKEN_HERE]"
openai_api_key = "[YOUR_DISCORD_BOT_TOKEN_HERE]"

# Verify that the environment variables have been set correctly.
if not discord_bot_token or not openai_api_key:
    raise ValueError("Environment variables for tokens are not set correctly.")

# Print the first few characters of the tokens for debugging purposes.
print(f"Discord Bot Token: {discord_bot_token[:5]}...")
print(f"OpenAI API Key: {openai_api_key[:5]}...")

# Define the channel ID where the bot will send an initial message.
TEST_CHANNEL = 1268919137242185859

# Configure the bot's intents to allow it to read messages and manage voice states.
intents = discord.Intents.default()
intents.messages = True  # Enable message-related events
intents.message_content = True  # Allow reading message content
intents.guilds = True  # Enable guild-related events
intents.voice_states = True  # Enable voice state updates
intents.reactions = True

# Initialize the bot with a command prefix and the specified intents.
bot = commands.Bot(command_prefix="-", intents=intents)
bot.remove_command("help")


@bot.event
async def on_ready():
    # Event triggered when the bot has successfully connected and is ready to operate.
    print("Bot is ready!")
    channel = bot.get_channel(TEST_CHANNEL)
    if channel:
        # Send a message to the specified channel to indicate the bot is active.
        await channel.send("Anime Thighs! has been turned on!")
        print(f"Message sent to channel: {TEST_CHANNEL}")
    else:
        print(f"Channel with ID {TEST_CHANNEL} not found.")

    # Print all registered commands to confirm they are correctly loaded.
    print("Registered commands:")
    for command in bot.commands:
        print(f"- {command.name}")

    # Start a periodic task to check voice channels every 30 seconds.
    check_voice_channels.start()


@bot.event
async def on_command_error(ctx, error):
    # Event triggered when an error occurs during the execution of a command.
    if isinstance(error, commands.CommandNotFound):
        # Send a message if the command is not found.
        await ctx.send("Command not found. Please use a valid command.")
        print("Command not found.")
    else:
        # Send a message with the error details.
        await ctx.send(f"An error occurred: {error}")
        print(f"An error occurred: {error}")


@bot.command()
async def hello(ctx):
    # Command to send a greeting message including the sender's name and ID.
    print("Received hello command")
    await ctx.send(f"Hello {ctx.author} ({ctx.author.id})!")


@bot.command()
async def ping(ctx):
    # Command to check the bot's latency and send it as a message.
    print("Received ping command")
    # Convert latency from seconds to milliseconds.
    latency = bot.latency * 1000
    await ctx.send(f"Kanga's Bot's Ping is {latency:.2f} ms")

# Dictionaries to manage voice clients, song queues, and played songs.
queues = {}  # Dictionary to store song queues for each guild
voice_clients = {}  # Dictionary to store voice clients for each guild
played_songs = {}  # Dictionary to keep track of played songs
currently_playing = {}  # Dictionary to keep track of the currently playing song

# Base URL and options for YouTube video extraction.
youtube_base_url = 'https://www.youtube.com/'
youtube_results_url = youtube_base_url + 'results?'
youtube_watch_url = youtube_base_url + 'watch?v='
yt_dl_options = {"format": "bestaudio/best"}
# Create an instance of YoutubeDL with options
ytdl = ytdlp.YoutubeDL(yt_dl_options)
ffmpeg_options = {'before_options': '-reconnect 1 -reconnect_streamed 1 -reconnect_delay_max 5',
                  'options': '-vn -filter:a "volume=0.25"'}


@bot.command()
async def play(ctx, *, link):
    # Command to play a YouTube video in the user's voice channel.
    try:
        # Get the voice channel of the user who invoked the command.
        voice_channel = ctx.author.voice.channel
        if not voice_channel:
            await ctx.send("You need to be in a voice channel to play music.")
            print("User not in a voice channel.")
            return

        # Connect to the voice channel if not already connected.
        if ctx.guild.id in voice_clients:
            voice_client = voice_clients[ctx.guild.id]
        else:
            print(f"Attempting to join {voice_channel}...")
            voice_client = await voice_channel.connect()
            voice_clients[ctx.guild.id] = voice_client
            print(f"Joined {voice_channel}")

        # If the link is not a YouTube URL, search for it.
        if youtube_base_url not in link:
            # Search for keywords and encode it
            query_string = urllib.parse.urlencode({'search_query': link})
            content = urllib.request.urlopen(
                youtube_results_url + query_string)  # Open the link
            search_results = re.findall(
                r'/watch\?v=(.{11})', content.read().decode())

            if not search_results:
                await ctx.send("No search results found.")
                print("No search results found.")
                return

            link = youtube_watch_url + search_results[0]  # Return ful link

        # Extract information from the YouTube link using yt-dlp.
        loop = asyncio.get_event_loop()
        data = await loop.run_in_executor(None, lambda: ytdl.extract_info(link, download=False))

        song_url = data['url']
        song_title = data.get('title', 'Unknown Title')
        player = discord.FFmpegPCMAudio(song_url, **ffmpeg_options)

        # Initialize the queue if not already done.
        if ctx.guild.id not in queues:
            queues[ctx.guild.id] = []
        if ctx.guild.id not in played_songs:
            played_songs[ctx.guild.id] = set()

        # Add the song to the queue if the bot is already playing.
        if voice_client.is_playing() or ctx.guild.id in currently_playing:
            # Add song url and title as tuple
            queues[ctx.guild.id].append((song_url, song_title))
            await ctx.send("Added to the queue.")
            print("Added to the queue.")
        else:
            # Start playing the song if nothing is currently playing.
            currently_playing[ctx.guild.id] = (song_url, song_title)
            voice_client.play(player, after=lambda e: asyncio.run_coroutine_threadsafe(
                play_next(ctx), bot.loop))
            await ctx.send(f"Now playing: {song_title}")
            print(f"Now playing: {song_title}")
            played_songs[ctx.guild.id].add(song_url)

    except Exception as e:
        # Send an error message if something goes wrong.
        await ctx.send(f"Error: {e}")
        print(f"Error: {e}")


@bot.command()
async def play_next(ctx):
    # Function to play the next song in the queue.
    try:
        user_voice_channel = ctx.author.voice.channel
        bot_voice_client = discord.utils.get(bot.voice_clients, guild=ctx.guild)

        if user_voice_channel is None:
            await ctx.send("You are not in a voice channel.")
            return

        if bot_voice_client is None or not bot_voice_client.is_connected():
            await ctx.send("Bot is not connected to a voice channel.")
            return

        if user_voice_channel.id != bot_voice_client.channel.id:
            await ctx.send("You must be in the same voice channel as the bot to perform this action.")
            return

        if ctx.guild.id in queues and queues[ctx.guild.id]:
            next_song_url, next_song_title = queues[ctx.guild.id].pop(0)
            currently_playing[ctx.guild.id] = (next_song_url, next_song_title)
            voice_client = voice_clients.get(ctx.guild.id)
            if voice_client:
                loop = asyncio.get_event_loop()
                data = await loop.run_in_executor(None, lambda: ytdl.extract_info(next_song_url, download=False))
                song_url = data['url']
                player = discord.FFmpegPCMAudio(song_url, **ffmpeg_options)
                voice_client.play(player, after=lambda e: asyncio.run_coroutine_threadsafe(
                    play_next(ctx), bot.loop))
                await ctx.send(f"Now playing: {next_song_title}")
                print(f"Now playing: {next_song_title}")
                played_songs[ctx.guild.id].add(next_song_url)
        else:
            currently_playing.pop(ctx.guild.id, None)
            print(f"Queue is empty for guild: {ctx.guild.id}")
    except Exception as e:
        await ctx.send(f"Error: {e}")
        print(f"Error: {e}")


@bot.command()
async def resume(ctx):
    # Command to resume the paused audio.
    print("Received resume command")
    try:
        user_voice_channel = ctx.author.voice.channel
        bot_voice_client = discord.utils.get(bot.voice_clients, guild=ctx.guild)

        if user_voice_channel is None:
            await ctx.send("You are not in a voice channel.")
            return

        if bot_voice_client is None or not bot_voice_client.is_connected():
            await ctx.send("Bot is not connected to a voice channel.")
            return

        if user_voice_channel.id != bot_voice_client.channel.id:
            await ctx.send("You must be in the same voice channel as the bot to perform this action.")
            return

        if ctx.guild.id in voice_clients and voice_clients[ctx.guild.id].is_paused():
            voice_clients[ctx.guild.id].resume()
            await ctx.send("Playback resumed.")
            print("Playback resumed.")
        else:
            await ctx.send("No audio is currently paused.")
            print("No audio is currently paused.")
    except Exception as e:
        # Send an error message if something goes wrong.
        await ctx.send(f"Error: {e}")
        print(f"Error: {e}")


@bot.command()
async def pause(ctx):
    # Command to stop the currently playing audio.
    try:
        user_voice_channel = ctx.author.voice.channel
        bot_voice_client = discord.utils.get(bot.voice_clients, guild=ctx.guild)

        if user_voice_channel is None:
            await ctx.send("You are not in a voice channel.")
            return

        if bot_voice_client is None or not bot_voice_client.is_connected():
            await ctx.send("Bot is not connected to a voice channel.")
            return

        if user_voice_channel.id != bot_voice_client.channel.id:
            await ctx.send("You must be in the same voice channel as the bot to perform this action.")
            return

        if ctx.guild.id in voice_clients and voice_clients[ctx.guild.id].is_playing():
            voice_clients[ctx.guild.id].pause()
            await ctx.send("Playback stopped.")
            print("Playback stopped.")
        else:
            await ctx.send("No audio is currently playing.")
            print("No audio is currently playing.")
    except Exception as e:
        # Send an error message if something goes wrong.
        await ctx.send(f"Error: {e}")
        print(f"Error: {e}")


@bot.command()
async def queue(ctx, *, url):
    # Command to add a song URL to the queue.
    try:
        if ctx.guild.id not in queues:
            queues[ctx.guild.id] = []
        queues[ctx.guild.id].append(url)
        # Update played_songs to allow re-adding of previously played songs
        if ctx.guild.id not in played_songs:
            played_songs[ctx.guild.id] = set()
        if url in played_songs[ctx.guild.id]:
            # Remove song that has been played
            played_songs[ctx.guild.id].remove(url)
        await ctx.send("Added to queue!")
    except Exception as e:
        # Send an error message if something goes wrong.
        print(f"Error: {e}")
        await ctx.send(f"Error: {e}")


@bot.command(name="clear_queue")
async def clear_queue(ctx):
    # Command to clear the song queue.
    try:
        user_voice_channel = ctx.author.voice.channel
        bot_voice_client = discord.utils.get(bot.voice_clients, guild=ctx.guild)

        if user_voice_channel is None:
            await ctx.send("You are not in a voice channel.")
            return

        if bot_voice_client is None or not bot_voice_client.is_connected():
            await ctx.send("Bot is not connected to a voice channel.")
            return

        if user_voice_channel.id != bot_voice_client.channel.id:
            await ctx.send("You must be in the same voice channel as the bot to perform this action.")
            return

        if ctx.guild.id in queues:
            queues[ctx.guild.id].clear()
            await ctx.send("Queue has been cleared.")
            print("Queue cleared.")
        else:
            await ctx.send("No songs in queue to be cleared.")
    except Exception as e:
        # Send an error message if something goes wrong.
        await ctx.send(f"Error: {e}")
        print(f"Error: {e}")


@bot.command(name="skip")
async def skip(ctx):
    # Command to skip the current track and play the next one in the queue.
    try:
        user_voice_channel = ctx.author.voice.channel
        bot_voice_client = discord.utils.get(bot.voice_clients, guild=ctx.guild)

        if user_voice_channel is None:
            await ctx.send("You are not in a voice channel.")
            return

        if bot_voice_client is None or not bot_voice_client.is_connected():
            await ctx.send("Bot is not connected to a voice channel.")
            return

        if user_voice_channel.id != bot_voice_client.channel.id:
            await ctx.send("You must be in the same voice channel as the bot to perform this action.")
            return

        if ctx.guild.id in voice_clients and voice_clients[ctx.guild.id].is_playing():
            voice_clients[ctx.guild.id].stop()
            await play_next(ctx)
            await ctx.send("Skipped current track!")
            print("Skipped current track")
        else:
            await ctx.send("No songs in queue to skip track.")
            print("No songs in queue to be able to skip.")
    except Exception as e:
        # Send an error message if something goes wrong.
        await ctx.send(f"Error: {e}")
        print(f"Error: {e}")


@bot.command(name="queue_list")
async def queue_list(ctx):
    # Command to display the list of songs currently in the queue.
    guild_id = ctx.guild.id

    try:
        currently_playing_message = ""
        if guild_id in currently_playing:
            song_url, song_title = currently_playing[guild_id]
            currently_playing_message = f"{song_title} (Currently Playing)\n"

        # Retrieve the list of songs in the queue
        if guild_id in queues and queues[guild_id]:
            queue_message = ""
            for i, (song_url, song_title) in enumerate(queues[guild_id], start=1):
                queue_message += f"{i}. {song_title}\n"
            await ctx.send(currently_playing_message + queue_message)
        else:
            await ctx.send(currently_playing_message + "The queue is currently empty.")
    except Exception as e:
        # Send an error message if something goes wrong.
        print(f"Error: {e}")
        await ctx.send(f"Error: {e}")


@tasks.loop(seconds=300)  # Periodically check every 30 seconds.
async def check_voice_channels():
    # Periodically check voice channels to disconnect if only the bot is present.
    for guild_id, voice_client in voice_clients.items():
        if voice_client.is_connected():
            channel = voice_client.channel
            # Only the bot is left in the channel.
            if len(channel.members) == 1:
                await voice_client.disconnect()
                if guild_id in queues:
                    queues[guild_id] = []
                print(f"Disconnected from {channel} as it was empty.")


@bot.command(name="del_song")
async def del_song(ctx, position: int):

    # Ensure the guild has a queue
    if ctx.guild.id not in queues:
        await ctx.send("Queue does not exist for this guild.")
        print("Queue does not exist for this guild.")
        return

    # Ensure the position is within the valid range
    if not (1 <= position <= len(queues[ctx.guild.id])):
        await ctx.send("Invalid position. Please provide a valid position.")
        print(f"Invalid position: {position}")
        return

    # Get the voice channel of the user and the bot
    user_voice_channel = ctx.author.voice.channel
    bot_voice_channel = voice_clients.get(ctx.guild.id)

    # Ensure the user is in the same voice channel as the bot
      user_voice_channel = ctx.author.voice.channel
      bot_voice_client = discord.utils.get(bot.voice_clients, guild=ctx.guild)

      if user_voice_channel is None:
          await ctx.send("You are not in a voice channel.")
          return

      if bot_voice_client is None or not bot_voice_client.is_connected():
          await ctx.send("Bot is not connected to a voice channel.")
          return

      if user_voice_channel.id != bot_voice_client.channel.id:
          await ctx.send("You must be in the same voice channel as the bot to perform this action.")
          return
    # Remove the song at the specified position (convert 1-based to 0-based index)
    try:
        removed_song = queues[ctx.guild.id].pop(position - 1)
        await ctx.send(f"Removed {removed_song[1]} from the queue.")
        print(f"Removed {removed_song[1]} from the queue.")
    except Exception as e:
        # Send an error message if something goes wrong.
        await ctx.send(f"Error: {e}")
        print(f"Error: {e}")

user_help_pages = {}

with open('Help.json', 'r') as file:
    help_pages = json.load(file)


def create_embed(page_content, user):
    embed = discord.Embed(
        colour=0x5865F2,
        title=page_content["title"],
        description=page_content["description"],
        timestamp=datetime.now()
    )

    embed.set_thumbnail(
        url='https://cdn.donmai.us/sample/0b/c6/__lucid_maplestory_drawn_by_muaooooo__sample-0bc65d4dc17171ed5104e89ceae935b5.jpg'
    )

    embed.set_image(
        url='https://cdn.donmai.us/sample/d6/b1/__lucid_maplestory_drawn_by_yoteh__sample-d6b1dfa7ec2c09867a4404ffa1730c97.jpg'
    )
    embed.set_author(name="Kanga's Bot",
                     icon_url='https://static.wikia.nocookie.net/maplestory/images/7/76/NPCArtwork_Lucid_%28Dream_Festa_5%29.png/revision/latest?cb=20231215172053')

    embed.set_footer(
        text="Made by @kangaroowastaken",
        icon_url=user.display_avatar.url
    )

    for field in page_content["fields"]:
        embed.add_field(
            name=field["name"],
            value=field["value"],
            inline=field["inline"]
        )

    return embed


@bot.command(name='help')
async def help_command(ctx, page: int = 1):
    Kangaroo_id = 572633005894402068
    user = await bot.fetch_user(Kangaroo_id)

    if 1 <= page <= len(help_pages):
        page_content = help_pages[page - 1]
        embed = create_embed(page_content, user)
        msg = await ctx.send(embed=embed)
        await msg.add_reaction("⬅️")
        await msg.add_reaction("➡️")

        # Store the message ID and current page number
        user_help_pages[msg.id] = {"page": page, "user_id": ctx.author.id}


@bot.event
async def on_reaction_add(reaction, user):
    if user.bot:
        return

    if reaction.message.id in user_help_pages:
        page_info = user_help_pages[reaction.message.id]

        if user.id != page_info["user_id"]:
            await reaction.remove(user=user)
            print("reaction removed")

        current_page = page_info["page"]
        new_page = current_page
        if reaction.emoji == "⬅️" and current_page > 1:
            new_page = current_page - 1
        elif reaction.emoji == "➡️" and current_page < len(help_pages):
            new_page = current_page + 1

        if new_page != current_page:
            page_info["page"] = new_page
            page_content = help_pages[new_page - 1]
            embed = create_embed(page_content, user)
            await reaction.message.edit(embed=embed)
        await reaction.remove(user=user)
        print('reaction removed')

# Store lobbies
lobbies = {}


class NimGame:
    def __init__(self, player1, player2, max_sticks=15, max_take=3):
        self.total_sticks = max_sticks
        self.max_take = max_take
        self.player1 = player1
        self.player2 = player2
        self.current_player = player1

    def remove_sticks(self, count):
        self.total_sticks -= count

    def switch_player(self):
        self.current_player = self.player2 if self.current_player == self.player1 else self.player1

    def is_game_over(self):
        return self.total_sticks <= 0


class NimView(View):
    def __init__(self, game, ctx):
        super().__init__()
        self.game = game
        self.ctx = ctx

    async def interaction_check(self, interaction):
        return interaction.user == self.game.current_player

    @discord.ui.button(label="Take 1", style=discord.ButtonStyle.primary)
    async def take_one(self, interaction: discord.Interaction, button: Button):
        await self.take_sticks(interaction, 1)

    @discord.ui.button(label="Take 2", style=discord.ButtonStyle.primary)
    async def take_two(self, interaction: discord.Interaction, button: Button):
        await self.take_sticks(interaction, 2)

    @discord.ui.button(label="Take 3", style=discord.ButtonStyle.primary)
    async def take_three(self, interaction: discord.Interaction, button: Button):
        await self.take_sticks(interaction, 3)

    async def take_sticks(self, interaction, count):
        self.game.remove_sticks(count)
        if self.game.is_game_over():
            await self.ctx.send(f"{interaction.user.mention} took the last stick! {interaction.user.mention} wins!")
            await self.ctx.send("Do you want to replay?", view=ReplayView(self.ctx, self.game))
            self.stop()
        else:
            self.game.switch_player()
            await interaction.response.edit_message(content=f"{self.game.current_player.mention}'s turn. Sticks remaining: {self.game.total_sticks}")


class ReplayView(View):
    def __init__(self, ctx, game):
        super().__init__()
        self.ctx = ctx
        self.game = game
        self.replay_votes = {game.player1.id: False, game.player2.id: False}

    async def check_replay_votes(self):
        if all(self.replay_votes.values()):
            await self.ctx.send("Both players agreed! Starting a new game...")
            await start_nim_game(self.ctx, self.game.player1, self.game.player2)

    @discord.ui.button(label="Replay", style=discord.ButtonStyle.success)
    async def replay(self, interaction: discord.Interaction, button: Button):
        self.replay_votes[interaction.user.id] = True
        await interaction.response.send_message(f"{interaction.user.mention} voted to replay.", ephemeral=True)
        await self.check_replay_votes()

    @discord.ui.button(label="Exit", style=discord.ButtonStyle.danger)
    async def exit(self, interaction: discord.Interaction, button: Button):
        del lobbies[self.game.player1.id]
        await interaction.response.edit_message(content="Game ended. Lobby closed.", view=None)


@bot.command(name='nim')
async def nim_lobby(ctx, opponent: discord.Member = None):
    if opponent:
        if opponent.id in lobbies and lobbies[opponent.id]['status'] == 'waiting':
            lobbies[opponent.id]['opponent'] = ctx.author
            lobbies[opponent.id]['status'] = 'in_progress'
            await ctx.send(f"{ctx.author.mention} joined {opponent.mention}'s lobby. The game is starting!")
            await start_nim_game(ctx, lobbies[opponent.id]['host'], ctx.author)
        else:
            await ctx.send(f"{opponent.mention} doesn't have an open lobby or is already in a game.")
    else:
        if ctx.author.id not in lobbies:
            lobbies[ctx.author.id] = {
                'host': ctx.author.id, 'opponent': None, 'status': 'waiting'}
            await ctx.send(f"{ctx.author.mention} created a Nim game lobby. Waiting for an opponent...")
        else:
            await ctx.send(f"{ctx.author.mention}, you already have a lobby open.")


@bot.command(name='nim_cancel')
async def nim_cancel(ctx):
    if ctx.author.id in lobbies and lobbies[ctx.author.id]['host'] == ctx.author:
        del lobbies[ctx.author.id]
        await ctx.send(f"{ctx.author.mention}, your lobby has been canceled.")
    else:
        await ctx.send(f"{ctx.author.mention}, you don't have a lobby to cancel, or you are not the host.")


@bot.command(name='nim_leave')
async def nim_leave(ctx):
    for lobby_id, lobby in lobbies.items():
        if lobby['opponent'] == ctx.author:
            lobbies[lobby_id]['opponent'] = None
            lobbies[lobby_id]['status'] = 'waiting'
            await ctx.send(f"{ctx.author.mention}, you have left the game. The lobby is now waiting for a new opponent.")
            return
    await ctx.send(f"{ctx.author.mention}, you are not in any active game.")


async def start_nim_game(ctx, player1, player2):
    game = NimGame(player1, player2)
    await ctx.send(f"Nim game starting between {player1.mention} and {player2.mention}!")
    await ctx.send(f"{game.current_player.mention}'s turn. Sticks remaining: {game.total_sticks}", view=NimView(game, ctx))


@bot.command(name='nim_lobbies')
async def view_lobbies(ctx):
    if lobbies:
        embed = discord.Embed(title="Active Nim Lobbies", colour=0x5865F2)
        for lobby_id, lobby in lobbies.items():
            host = await bot.fetch_user(lobby["host"])
            opponent = lobby['opponent'].mention if lobby['opponent'] else "Waiting for opponent..."
            status = lobby['status'].capitalize()
            embed.add_field(name=f"Lobby by {host}", value=f"Opponent: {opponent}\nStatus: {status}", inline=False)
        await ctx.send(embed=embed)
    else:
        await ctx.send("There are no active lobbies at the moment.")


def open_database(ctx):
    """
    Opens the database file and returns the data. If the file doesn't exist,
    initializes it with default values for a new user.
    """
    try:
        with open('Database.json', 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        data = {}  # Initialize with an empty dictionary if file not found

    user_id = str(ctx.author.id)
    if user_id not in data:
        # Initialize new user data with default values
        data[user_id] = {
            'Wallet': 0,
            'Bank': 0,
            'Daily Timer': 0,
            'Daily Streak': 0,
            'Weekly Timer': 0,
            'Rob Timer': 0,
            'Beg Timer': 0,
            'Search Timer': 0,
            'Pray Timer': 0,
            'Status Effects': {},
            'Inventory': {}
        }
        with open('Database.json', 'w') as f:
            json.dump(data, f, indent=4)

    return data


def save_data(data):
    """
    Saves the provided data to the database file.
    """
    with open('Database.json', 'w') as f:
        json.dump(data, f, indent=4)

def remove_expired_status_effects(data, user_id):
    now = get_current_timestamp()
    status_effects = data.get(user_id, {}).get('Status Effects', {})
    if status_effects and now > status_effects.get('Expires', 0):
        data[user_id]['Status Effects'] = {}
        return True
    return False

async def check_cooldown(ctx, user_id, timer_key, cooldown_period, embed_title, embed_description):
    """
    Checks if the user is still under cooldown for a specified action.
    Sends an embed message if they are still on cooldown.
    """
    data = open_database(ctx)
    now = datetime.now(timezone.utc)
    current_timestamp = get_current_timestamp()

    last_timestamp = data[user_id].get(timer_key, 0)
    last_time = datetime.fromtimestamp(
        last_timestamp, tz=timezone.utc) if last_timestamp else now - cooldown_period

    if now - last_time < cooldown_period:
        next_time = last_timestamp + cooldown_period.total_seconds()
        embed = discord.Embed(
            title=embed_title,
            description=embed_description.format(int(next_time)),
            color=discord.Color.red()
        )
        await ctx.send(embed=embed)
        return False
    return True


@bot.command(name='daily')
async def daily_reward(ctx):
    """
    Allows users to claim their daily reward. The reward amount can be affected by streak bonuses.
    """
    data = open_database(ctx)
    user_id = str(ctx.author.id)
    user_data = data[user_id]
    user = await bot.fetch_user(ctx.author.id)

    now = datetime.now(timezone.utc)
    current_timestamp = int(now.timestamp())
    daily_cd = timedelta(days=1)  # 24-hour cooldown period
    streak_reset = timedelta(days=2)  # Reset streak after 2 days

    last_claim_timestamp = user_data.get('Daily Timer', 0)
    last_claim_time = datetime.fromtimestamp(
        last_claim_timestamp, tz=timezone.utc) if last_claim_timestamp else now - timedelta(days=2)

    if now - last_claim_time >= daily_cd:
        user_data['Daily Timer'] = current_timestamp

        if now - last_claim_time >= streak_reset:
            data[user_id]['Daily Streak'] = 0

        streak_bonus = 1 + min(data[user_id]['Daily Streak'] * 0.03, 3.0)
        random_reward = rand.randint(1000, 3000)
        total_amount = int(random_reward * streak_bonus)
        user_data['Wallet'] += total_amount
        data[user_id]['Daily Streak'] += 1

        save_data(data)

        embed = discord.Embed(
            title=f"{ctx.author.name}'s Daily Reward",
            description=f"You've claimed your daily reward of {total_amount} KanCoins!",color=discord.Color.green()
        )
        embed.set_footer(text=f"Daily Streak: {data[user_id]['Daily Streak']} | Bonus: {data[user_id]['Daily Streak'] * 0.03 * 100}%", icon_url=user.display_avatar.url)

        await ctx.send(embed=embed)
    else:
        next_claim_time = last_claim_time + daily_cd
        next_claim_timestamp = int(next_claim_time.timestamp())

        embed = discord.Embed(
            title=f"{ctx.author.name}'s Daily Reward Not Available",
            description=f"Your daily reward is not ready yet! Return back in <t:{next_claim_timestamp}:R>, <t:{next_claim_timestamp}:f>!",
            color=discord.Color.red()
        )
        embed.set_footer(text=f"Daily Streak: {data[user_id]['Daily Streak']} | Bonus: {data[user_id]['Daily Streak'] * 0.03 * 100}%", icon_url=user.display_avatar.url)

        await ctx.send(embed=embed)


@bot.command(name='bal')
async def check_balance(ctx):
    """
    Shows the user's current wallet and bank balance.
    """
    data = open_database(ctx)
    user = await bot.fetch_user(ctx.author.id)

    embed = discord.Embed(
        title=f"{ctx.author.name}'s Balance",
        color=discord.Color.blue()
    )
    embed.add_field(name="Wallet:", value=f"{data[str(ctx.author.id)]['Wallet']} KanCoins")
    embed.add_field(name="Bank Account:", value=f"{data[str(ctx.author.id)]['Bank']} KanCoins")
    embed.set_footer(text=f"Requested by {ctx.author.name}", icon_url=user.display_avatar.url)

    await ctx.send(embed=embed)


def get_current_timestamp():
    """
    Returns the current timestamp in seconds.
    """
    return int(datetime.now(timezone.utc).timestamp())


def apply_status_effects(data, user_id, amount, command_type=None):
    """
    Applies status effects to the amount of money.
    """
    status_effect = data[user_id]['Status Effects']

    if command_type in ['beg', 'search'] and status_effect.get('Blessing') == 'Demonic Blessing':
        return amount  # No multiplier applied for 'beg' and 'search' with Demonic Blessing

    if status_effect != {}:
        multiplier = status_effect['Multiplier']
        amount *= multiplier
    else:
        amount = amount
    return amount


@bot.command(name='rob')
async def rob_people(ctx, target: discord.User):
    """
    Allows users to rob another user with varying success chances based on status effects.
    """
    data = open_database(ctx)
    user_id = str(ctx.author.id)
    target_id = str(target.id)
    if remove_expired_status_effects(data, user_id):
        save_data(data)

    if not await check_cooldown(ctx, user_id, 'Rob Timer', timedelta(minutes=30),
                                f"{ctx.author.name}'s Rob Cooldown",
                                "Don't be so hasty, you can rob again in <t:{}:R>."):
        return

    if target_id not in data:
        await ctx.send("Target user does not have a balance record.")
        return

    if user_id not in data:
        await ctx.send("Your account does not have a balance record.")
        return

    if data[target_id]['Wallet'] <= 0:
        await ctx.send("They're too broke to be robbed.")
        return

    wallet_target = data[target_id].get('Wallet', 0)
    base_wallet_robbed = int(wallet_target * (rand.randint(2, 3) * 0.1))
    wallet_robbed = apply_status_effects(data, user_id, base_wallet_robbed)
    number = rand.randint(0, 10)

    chance_increment = 1
    status_effect = data[user_id].get('Status Effects', {})
    multiplier = 1
    fine_multiplier = 1

    if status_effect.get('Blessing') == 'Demonic Blessing':
        multiplier += 1  # 100% more for robbery
        chance_increment = 1.5
        fine_multiplier = 1.3  # 30% more if fined

    if number < int(2 * chance_increment):
        wallet_robbed = int(wallet_robbed * multiplier)
        data[target_id]['Wallet'] -= wallet_robbed
        data[user_id]['Wallet'] += wallet_robbed
        result_message = f"You've successfully robbed {target.name} for {wallet_robbed} KanCoin!"
        data[user_id]['Rob Timer'] = get_current_timestamp()

    elif number > 8:
        # Apply the fine multiplier
        fine = int(data[user_id]['Wallet'] * 0.2 * fine_multiplier)
        data[user_id]['Wallet'] -= fine
        result_message = f"You've been caught! Your fine was {fine} KanCoin."
        data[user_id]['Rob Timer'] = get_current_timestamp()

    else:
        result_message = f"You've failed to rob {target.name}!"
        data[user_id]['Rob Timer'] = get_current_timestamp()

    save_data(data)
    await ctx.send(result_message)


@bot.command(name='beg')
async def beggars_choosers(ctx):
    """
    Allows users to beg for money with varying amounts based on chance and status effects.
    """
    data = open_database(ctx)
    user_id = str(ctx.author.id)

    if remove_expired_status_effects(data, user_id):
        save_data(data)

    if not await check_cooldown(ctx, user_id, 'Beg Timer', timedelta(seconds=15),
                                f"{ctx.author.name}'s Beg Cooldown",
                                "Calm down broke boy, you can beg again <t:{}:R>."):
        return

    number = rand.randint(0, 10)
    beg_money = 0

    if number > 9:
        beg_money = int(rand.randint(1000, 2000) * 1.9)
    elif number < 1:
        beg_money = int(rand.randint(500, 1000) * 1.1)
    else:
        beg_money = int(rand.randint(100, 1000) * 1.3)

    beg_money = apply_status_effects(data, user_id, beg_money, 'beg')

    data[user_id]['Wallet'] += beg_money
    data[user_id]['Beg Timer'] = get_current_timestamp()

    save_data(data)

    embed = discord.Embed(
        title=f"{ctx.author.name}'s Begging Results",
        description=f"You've begged for {beg_money} KanCoins!",
        color=discord.Color.green()
    )
    await ctx.send(embed=embed)


def pray_to_demon():
    chance = rand.random()
    if chance < 0.1:  # 10% chance for a curse
        return "Wrath of the Underworld", 0.3, timedelta(days=1)
    elif chance < 0.2:  # 10% chance for a blessing
        return "Demonic Blessing", 2, timedelta(hours=3)
    else:
        return "I have no mouth and I must scream", 0.8, timedelta(days=1)


def pray_to_god():
    chance = rand.random()
    if chance < 0.05:  # 5% chance of angering the gods
        return "Divine Retribution", 0.8, timedelta(hours=12)
    elif chance < 0.25:  # 20% chance of pleasing the gods
        return "Divine Blessing", 1.25, timedelta(hours=12)
    else:
        return "Devotee", 1.05, timedelta(days=1)
# Pray command


@bot.command(name='pray')
async def pray(ctx, choice: str):
    data = open_database(ctx)
    user_id = str(ctx.author.id)
    now = datetime.now(timezone.utc)
    current_timestamp = get_current_timestamp()
    pray_cd = timedelta(days=1)
    if remove_expired_status_effects(data, user_id):
        save_data(data)

    if not await check_cooldown(ctx, user_id, 'Pray Timer', pray_cd,
                                f"{ctx.author.name}'s Pray Cooldown",
                                "Calm down, you can pray again <t:{}:R>."):
        return

    if choice.lower() == 'demon':
        blessing, multiplier, duration = pray_to_demon()
        data[user_id]['Pray Timer'] = current_timestamp
    elif choice.lower() == 'god':
        blessing, multiplier, duration = pray_to_god()
        data[user_id]['Pray Timer'] = current_timestamp
    else:
        await ctx.send("Please choose either 'demon' or 'god'.")
        return

    expiration_time = current_timestamp + int(duration.total_seconds())
    data[user_id]['Status Effects'] = {
        "Blessing": blessing,
        "Multiplier": multiplier,
        "Expires": expiration_time
    }

    save_data(data)

    embed = discord.Embed(
        title=f"{ctx.author.name}'s Prayer Result",
        description=f"**{blessing}** applied! Multiplier: {multiplier}. Duration: <t:{expiration_time}:R>.",color=discord.Color.green()
    )
    await ctx.send(embed=embed)

# Command to check Status


@bot.command(name='check_status')
async def check_status(ctx):
    data = open_database(ctx)
    user_id = str(ctx.author.id)
    if remove_expired_status_effects(data, user_id):
        save_data(data)

    status_effect = data[user_id].get('Status Effects', {})

    if status_effect:
        expiration_time = status_effect['Expires']
        embed = discord.Embed(
            title=f"{ctx.author.name}'s Status",
            description=f"**{status_effect['Blessing']}** active! Multiplier: {status_effect['Multiplier']}. Expires in <t:{expiration_time}:R>.",
            color=discord.Color.blue()
        )
        await ctx.send(embed=embed)
    else:
        await ctx.send("You have no active status effects.")

# Background task to check status effects every hour

@bot.command(name='search')
async def search(ctx):
    """
    Allows users to search for money with varying success rates and amounts.
    """
    data = open_database(ctx)
    user_id = str(ctx.author.id)
    if remove_expired_status_effects(data, user_id):
        save_data(data)

    if not await check_cooldown(ctx, user_id, 'Search Timer', timedelta(seconds=30),
                                f"{ctx.author.name}'s Search Cooldown",
                                "You can search again in <t:{}:R>."):
        return

    number = rand.randint(0, 10)
    search_money = 0

    if number > 9:
        search_money = int(rand.randint(1000, 2000) * 1.5)
    elif number < 1:
        search_money = int(rand.randint(500, 1000) * 1.2)
    else:
        search_money = int(rand.randint(100, 500) * 1.1)

    search_money = apply_status_effects(data, user_id, search_money, 'search')

    data[user_id]['Wallet'] += search_money
    data[user_id]['Search Timer'] = get_current_timestamp()

    save_data(data)

    embed = discord.Embed(
        title=f"{ctx.author.name}'s Search Results",
        description=f"You've found {search_money} KanCoins!",
        color=discord.Color.green()
    )
    await ctx.send(embed=embed)

user_shop_pages = {}

with open('shop.json', 'r') as file:
    shop_pages = json.load(file)


def create_embed(page_content, user):
    embed1 = discord.Embed(
        colour=0x5865F2,
        title=page_content["title"],
        description=page_content["description"],
        timestamp=datetime.now()
    )

    embed1.set_thumbnail(
        url='https://cdn.donmai.us/sample/0b/c6/__lucid_maplestory_drawn_by_muaooooo__sample-0bc65d4dc17171ed5104e89ceae935b5.jpg'
    )

    embed1.set_image(
        url='https://cdn.donmai.us/sample/d6/b1/__lucid_maplestory_drawn_by_yoteh__sample-d6b1dfa7ec2c09867a4404ffa1730c97.jpg'
    )
    embed1.set_author(name="Kanga's Bot",
                     icon_url='https://static.wikia.nocookie.net/maplestory/images/7/76/NPCArtwork_Lucid_%28Dream_Festa_5%29.png/revision/latest?cb=20231215172053')

    embed1.set_footer(
        text="Made by @kangaroowastaken",
        icon_url=user.display_avatar.url
    )

    for field in page_content["fields"]:
        embed1.add_field(
            name=field["name"],
            value=field["value"],
            inline=field["inline"]
        )

    return embed1

@bot.command(name='shop')
async def help_command(ctx, page: int = 1):
    Kangaroo_id = 572633005894402068
    user = await bot.fetch_user(Kangaroo_id)

    if 1 <= page <= len(shop_pages):
        page_content = shop_pages[page - 1]
        embed1 = create_embed(page_content, user)
        msg = await ctx.send(embed=embed1)
        await msg.add_reaction("⬅️")
        await msg.add_reaction("➡️")

        # Store the message ID and current page number
        user_shop_pages[msg.id] = {"page": page, "user_id": ctx.author.id}

@bot.event
async def on_reaction_add(reaction, user):
    if user.bot:
        return

    if reaction.message.id in user_shop_pages:
        page_info = user_shop_pages[reaction.message.id]

        if user.id != page_info["user_id"]:
            await reaction.remove(user=user)
            print("reaction removed")

        current_page = page_info["page"]
        new_page = current_page
        if reaction.emoji == "⬅️" and current_page > 1:
            new_page = current_page - 1
        elif reaction.emoji == "➡️" and current_page < len(shop_pages):
            new_page = current_page + 1

        if new_page != current_page:
            page_info["page"] = new_page
            page_content = shop_pages[new_page - 1]
            embed = create_embed(page_content, user)
            await reaction.message.edit(embed=embed)
        await reaction.remove(user=user)
        print('reaction removed')

@bot.event
async def on_message(message):
    # Event triggered when a message is received.
    print(f"Message from {message.author} in {message.channel}: {message.content}")
    if message.author == bot.user:
        return
    await bot.process_commands(message)  # Process commands in the message.

# Start the bot with the specified token.
nest_asyncio.apply()
bot.run(discord_bot_token)

2024-08-24 17:45:41 INFO     discord.client logging in using static token
2024-08-24 17:45:41 INFO     discord.client logging in using static token
2024-08-24 17:45:41 INFO     discord.client logging in using static token
2024-08-24 17:45:41 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token


Discord Bot Token: ODYyM...
OpenAI API Key: sk-pr...


2024-08-24 17:45:41 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: e16fd4d070e338e2440689afcd24df3e).
2024-08-24 17:45:41 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: e16fd4d070e338e2440689afcd24df3e).
2024-08-24 17:45:41 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: e16fd4d070e338e2440689afcd24df3e).
2024-08-24 17:45:41 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: e16fd4d070e338e2440689afcd24df3e).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: e16fd4d070e338e2440689afcd24df3e).


Message from kangaroowastaken in botcommands: -search
Message from Kanga's Bot#2803 in botcommands: 
Bot is ready!
Message from Kanga's Bot#2803 in test: Anime Thighs! has been turned on!
Message sent to channel: 1268919137242185859
Registered commands:
- queue
- search
- clear_queue
- help
- hello
- shop
- skip
- nim
- ping
- play_next
- play
- pray
- rob
- bal
- resume
- del_song
- queue_list
- stop
- check_status
- nim_cancel
- nim_leave
- beg
- daily
- nim_lobbies
Message from kangaroowastaken in botcommands: -beg
Message from Kanga's Bot#2803 in botcommands: 
Message from kangaroowastaken in botcommands: -bal
Message from Kanga's Bot#2803 in botcommands: 
Message from kangaroowastaken in botcommands: -daily
Message from Kanga's Bot#2803 in botcommands: 
Message from kangaroowastaken in general: 
